In [279]:
import numpy as np
import pandas as pd
import tensorflow as tf
from urllib.request import urlopen
import certifi
import ssl
from bs4 import BeautifulSoup
import requests

from sklearn.model_selection import train_test_split
context = ssl.create_default_context(cafile=certifi.where())

In [280]:


# define urls for per game and advanced stats
#extracting data from site
def get_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    
 
    data = []
    if url[-13:]=="advanced.html":
        headers = [th.text.strip() for th in table.find_all('th') if th.text.strip()!=""][7:]
        for tr in table.find_all('tr')[1:]:
            if str(tr)[:16]=="<tr class=\"thead" or str(tr)[:16]=="<tr class=\"itali":
                continue
            row = np.array([td.text.strip() for td in tr.find_all('td')[6:] if "DUMMY" not in str(td)])
            data.append(row)
    else:
        headers = [th.text.strip() for th in table.find_all('th')]
        
        for tr in table.find_all('tr')[1:]:
            
            if str(tr)[:16]=="<tr class=\"thead"or str(tr)[:16]=="<tr class=\"itali" :
                
                if data[-1][4]=="TOT":
                    data[-1][4]=[td.text.strip() for td in tr.find_all('td') ][3]
                    
                continue
            
            
            row = np.append([0],np.array([td.text.strip() for td in tr.find_all('td') ]))
            
            
            data.append(row)
          
    headers=headers[:row.size]
    arr=np.array(data)
      

    return arr,headers


In [283]:
#creating data frame

per_game_url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'
advanced_url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'
def data_frame():
    for i in range(1997,2024):
        per_game_for_seas,per_game_headers=get_data(per_game_url.format(i))
        per_game_for_seas=np.insert(per_game_for_seas,3,i,axis=1)
        advanced_for_seas,adnvanced_headers=get_data(advanced_url.format(i))
        temp_full=np.concatenate((per_game_for_seas,advanced_for_seas),axis=1)
        if i==1997:
            full=temp_full
        else:
            full=np.concatenate((full,temp_full))
        print(i)
        
    whole_data=pd.DataFrame(full)
    headers=np.concatenate((per_game_headers,adnvanced_headers))
    headers=np.insert(headers, 3,"Season")
    whole_data.columns=headers
    whole_data.Rk=[i for i in range(1,len(full)+1)]
    return whole_data

   
   

In [284]:
whole_data=data_frame()


nba_abbreviations_teams = {
    'PHO': 'Phoenix Suns',
    'WSB':'Washington Wizards',
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BKN': 'Brooklyn Nets',
    'BRK':'Brooklyn Nets',
    'CHO':'Charlotte Hornets',
    'CHH': 'Charlotte Hornets',
    'NOK': 'New Orleans Pelicans',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHX': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards',
    'CHA': 'Charlotte Hornets', # Became Charlotte Hornets in 2014
    'NJN': 'New Jersey Nets',   # Moved to Brooklyn and became Brooklyn Nets in 2012
    'NOH': 'New Orleans Hornets', # Became New Orleans Pelicans in 2013
    'SEA': 'Seattle SuperSonics', # Relocated to Oklahoma City and became Oklahoma City Thunder in 2008
    'VAN': 'Memphis Grizzlies', # Relocated to Memphis and became Memphis Grizzlies in 2001
}

1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [285]:
whole_data

,Rk,Player,Pos,Season,Age,Tm,G,GS,MP,FG,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,1,Mahmoud Abdul-Rauf,PG,1997,27,SAC,75,51,28.4,5.5,...,10.8,23.2,2.0,0.7,2.7,.060,-0.8,-1.8,-2.6,-0.4
1,2,Shareef Abdur-Rahim,PF,1997,20,VAN,80,71,35.0,6.9,...,13.5,27.6,1.7,1.2,2.9,.049,0.3,-2.1,-1.8,0.2
2,3,Rafael Addison,SF,1997,32,CHH,41,3,8.7,1.2,...,11.2,20.1,0.2,0.1,0.4,.048,-2.6,-1.7,-4.4,-0.2
3,4,Cory Alexander,PG,1997,23,SAS,80,6,18.2,2.4,...,21.1,22.2,0.5,0.6,1.1,.035,0.3,-0.8,-0.6,0.5
4,5,Jerome Allen,SG,1997,24,IND,76,1,12.4,1.0,...,21.4,15.6,-0.3,0.8,0.4,.021,-2.5,-0.2,-2.6,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12841,12842,Thaddeus Young,PF,2023,34,TOR,54,9,14.7,2.0,...,16.7,13.5,0.7,1.1,1.8,.109,-1.8,1.9,0.1,0.4
12842,12843,Trae Young,PG,2023,24,ATL,73,73,34.8,8.2,...,15.2,32.6,5.3,1.4,6.7,.126,5.3,-2.0,3.3,3.4
12843,12844,Omer Yurtseven,C,2023,24,MIA,9,0,9.2,1.8,...,11.9,18.0,0.2,0.1,0.3,.159,-2.5,-1.5,-3.9,0.0
12844,12845,Cody Zeller,C,2023,30,MIA,15,2,14.5,2.5,...,15.8,18.1,0.4,0.3,0.7,.147,-2.0,-0.7,-2.8,0.0


In [286]:
whole_data.to_csv("whole_data.csv")

numerical_data=pd.DataFrame(whole_data.values[:,6:],columns=whole_data.columns[6:])



In [287]:
p_id=pd.DataFrame(np.zeros((whole_data.shape[0],8)),columns=["Rk","Player","Age", "season","team","seed","cons_mvp","mvp_votes"])
p_id.Rk=whole_data.Rk
p_id.Age=whole_data.Age
p_id.Player=whole_data.Player
p_id.season=whole_data.Season
p_id.team=whole_data.Tm



In [288]:
def givehtml(s):
    wikiurl="https://en.wikipedia.org/wiki/{}%E2%80%93{}_NBA_season"
    if s<2009 and s>=1999:
        page=urlopen(wikiurl.format(str(s),"0"+str((s+1)%100)))
    
    else:
        page=urlopen(wikiurl.format(str(s),str((s+1)%100)))
    htmllang=page.read()
    htmlstr=htmllang.decode("utf-8")
    

    return htmlstr


    
def cut_data(htmlstr):
    
    startEastv1=htmlstr.index("<span class=\"mw-headline\" id=\"By_conference\">By conference</span>")
    startEast=htmlstr[startEastv1:].index("season")+1+startEastv1
    finishEast=htmlstr[startEast:].index("West")+startEast
    startWest=finishEast
    finishWest=htmlstr[startWest:].index("Playoffs")+startWest
    east_data=htmlstr[startEast:finishEast]
    west_data=htmlstr[startWest:finishWest]

    return east_data,west_data


def get_stands(results):
    for i in range(1996,2023):
     try:
        east_data,west_data=cut_data(givehtml(i))
        res_dictionary={}
        #east
       
        start=east_data.find("season")
        n=30
        while start >= 0 and n > 1:
            start = east_data.find("season", start+1)
            if n%2==0:
                  sign=east_data[start:].find("<")+start
                  res_dictionary[east_data[start+8:sign]]=int((32-n)/2)
            n -= 1
        
        #west
        if i>=2004:
            start=west_data.find("season")
            n=30
            while start >= 0 and n > 1:
                start = west_data.find("season", start+1)
                if n%2==0:
                  sign=west_data[start:].find("<")+start
                  res_dictionary[west_data[start+8:sign]]=int((32-n)/2)
                n-=1
        else:
            n=28
            start=west_data.find("season")
      
            while start >= 0 and n > 1:
                start = west_data.find("season", start+1)
                if n%2==0:
                  sign=west_data[start:].find("<")+start
                  res_dictionary[west_data[start+8:sign]]=int((30-n)/2)
                n-=1
        results.append(res_dictionary)        

     except: return i
    
        

In [289]:
#changing past team names
results=[] 
get_stands(results)


for i in results:
    try:
       val=i["Washington Bullets"]
       del i["Washington Bullets"]
       i["Washington Wizards"]=val
    except:
         pass
    
    try:
        val=i['Vancouver Grizzlies']
        del i['Vancouver Grizzlies']
        i['Memphis Grizzlies']=val
    except:pass
    try:
        val=i['Charlotte Bobcats']
        del i['Charlotte Bobcats']
        i['Charlotte Hornets']=val
    except:pass
    try:
        val=i['New Orleans/Oklahoma City Hornets']
        del i['New Orleans/Oklahoma City Hornets']
        i['New Orleans Pelicans']=val
    except:pass
    
   
#seeding
new_seed=np.zeros_like(p_id.seed)
for i,v in enumerate(p_id.values):
       
        
        new_seed[i]=int(results[int(v[3])-1997][nba_abbreviations_teams[v[4]]])
        

        


p_id.seed=new_seed

In [290]:
#scraping data
def get_mvp_vote_data(url,mvp_data_storage):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
 
    this_year_votes={}
    current=""
    column=0
    for td in table.find_all('td'):
        
        if column%19==0:
            current=td.text.strip()
        elif column%19==4:
            points=float(td.text.strip())
            this_year_votes[current]=points
        column+=1
    mvp_data_storage.append(this_year_votes)

mvp_votes_url="https://www.basketball-reference.com/awards/awards_{}.html"
def gather_mvp_voting():
    mvp_data_storage=[]
    for i in range(1997,2024):
        get_mvp_vote_data(mvp_votes_url.format(i),mvp_data_storage)
    return mvp_data_storage
        
mvp_voting_data=gather_mvp_voting()
print(mvp_voting_data)

[{'Karl Malone': 986.0, 'Michael Jordan': 957.0, 'Grant Hill': 376.0, 'Tim Hardaway': 238.0, 'Glen Rice': 134.0, 'Gary Payton': 105.0, 'Hakeem Olajuwon': 95.0, 'Patrick Ewing': 57.0, 'Anthony Mason': 7.0, "Shaquille O'Neal": 7.0, 'Scottie Pippen': 6.0, 'Alonzo Mourning': 5.0, 'Dikembe Mutombo': 4.0, 'Mitch Richmond': 4.0, 'John Stockton': 3.0, 'Charles Barkley': 2.0, 'Tom Gugliotta': 1.0, 'Allen Iverson': 1.0, 'Kevin Johnson': 1.0, 'Steve Smith': 1.0}, {'Michael Jordan': 1084.0, 'Karl Malone': 842.0, 'Gary Payton': 431.0, "Shaquille O'Neal": 311.0, 'Tim Duncan': 148.0, 'Tim Hardaway': 71.0, 'David Robinson': 36.0, 'Vin Baker': 24.0, 'Grant Hill': 23.0, 'Scottie Pippen': 14.0, 'Glen Rice': 7.0, 'Antoine Walker': 6.0, 'Jason Kidd': 5.0, 'John Stockton': 5.0, 'Mitch Richmond': 3.0, 'Reggie Miller': 2.0, 'Rik Smits': 2.0, 'Michael Finley': 1.0, 'Rod Strickland': 1.0}, {'Karl Malone': 827.0, 'Alonzo Mourning': 773.0, 'Tim Duncan': 740.0, 'Allen Iverson': 319.0, 'Jason Kidd': 159.0, "Shaquil

In [291]:
cur_year=0.0
mvp=""
cons=0.0
for i,v in enumerate(p_id.values):

    if v[3]!=cur_year:
        cur_year=v[3]
        cur_year_data=mvp_voting_data[int(cur_year)-1997]
        max_val=max(cur_year_data.values())
    if v[1] in cur_year_data.keys():
        p_id.at[i,"mvp_votes"]=cur_year_data[v[1]]
       
        if p_id.at[i,"mvp_votes"]==max_val:
            if p_id.at[i,"Player"]== mvp:
                print(p_id.at[i,"Player"])
                cons+=1
            else:
                cons=0
            mvp=p_id.at[i,"Player"]
            p_id.at[i,"cons_mvp"]=cons




p_id.to_csv("p_id.csv")

LeBron James
LeBron James
Stephen Curry
Giannis Antetokounmpo
Nikola Jokić


In [292]:
p_id

,Rk,Player,Age,season,team,seed,cons_mvp,mvp_votes
0,1,Mahmoud Abdul-Rauf,27,1997,SAC,9.0,0.0,0.0
1,2,Shareef Abdur-Rahim,20,1997,VAN,14.0,0.0,0.0
2,3,Rafael Addison,32,1997,CHH,6.0,0.0,0.0
3,4,Cory Alexander,23,1997,SAS,13.0,0.0,0.0
4,5,Jerome Allen,24,1997,IND,10.0,0.0,0.0
...,...,...,...,...,...,...,...,...
12841,12842,Thaddeus Young,34,2023,TOR,9.0,0.0,0.0
12842,12843,Trae Young,24,2023,ATL,8.0,0.0,0.0
12843,12844,Omer Yurtseven,24,2023,MIA,7.0,0.0,0.0
12844,12845,Cody Zeller,30,2023,MIA,7.0,0.0,0.0


In [293]:
combined=pd.DataFrame(np.concatenate((p_id.drop(["Player","season","team"],axis=1).values,numerical_data),axis=1),columns=np.append(p_id.drop(["Player","season","team"],axis=1).columns,numerical_data.columns))

for i in combined:
    if i!="Rk" :
        for k,n in enumerate(combined[i]):
                if n=="":
                        combined.at[k,i]=0.0
                
for i in combined:
        if i!="Rk":
                combined[i]=np.array(combined[i], dtype=float)
                combined[i]=np.array((combined[i]-min(combined[i]))/(max(combined[i])-min(combined[i])))
                

y=combined.mvp_votes
x=combined.drop(["mvp_votes"],axis=1)


In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
for i,v in enumerate(y_pred):
    if v>0.1:
        print(y_pred)


Mean Squared Error: 0.0012463213214104082
[-0.00228697 -0.00216861  0.00665421 ...  0.0010478   0.00128555
  0.00100224]
[-0.00228697 -0.00216861  0.00665421 ...  0.0010478   0.00128555
  0.00100224]
[-0.00228697 -0.00216861  0.00665421 ...  0.0010478   0.00128555
  0.00100224]
[-0.00228697 -0.00216861  0.00665421 ...  0.0010478   0.00128555
  0.00100224]


In [310]:



from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(x.drop("cons_mvp",axis=1), y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

# Get coefficients
coefficients = lr_model.coef_

# Create a DataFrame to store feature names and coefficients
coef_df = pd.DataFrame({'Feature': x.drop("cons_mvp",axis=1).columns, 'Coefficient': coefficients})

# Sort features by the absolute value of their coefficients
coef_df['AbsoluteCoefficient'] = abs(coef_df['Coefficient'])
coef_df = coef_df.sort_values(by='Coefficient', ascending=False)

# Print feature importance ranking
print("Feature Importance Ranking:")
print(coef_df)
y_pred = lr_model.predict(x_test)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Feature Importance Ranking:
   Feature   Coefficient  AbsoluteCoefficient
46     BPM  2.085184e+00         2.085184e+00
47    VORP  2.505473e-01         2.505473e-01
20     DRB  1.596843e-01         1.596843e-01
10     3PA  1.311267e-01         1.311267e-01
15    eFG%  1.057712e-01         1.057712e-01
34    TRB%  1.007788e-01         1.007788e-01
25     TOV  8.556549e-02         8.556549e-02
27     PTS  7.198542e-02         7.198542e-02
28     PER  6.345316e-02         6.345316e-02
17     FTA  4.755750e-02         4.755750e-02
19     ORB  3.851325e-02         3.851325e-02
13     2PA  2.428991e-02         2.428991e-02
26      PF  2.288243e-02         2.288243e-02
42      WS  1.548131e-02         1.548131e-02
35    AST%  1.134443e-02         1.134443e-02
3        G  1.087555e-02         1.087555e-02
18     FT%  8.656340e-03         8.656340e-03
12      2P  3.018013e-03         3.018013e-03
0       Rk  4.541301e-07         4.541301e-07
6       FG -6.153507e-04         6.153507e-04
2     

In [313]:
def last(arr):
    return arr[-1]

bangers=[]
for i,v in enumerate(y_pred):
    if v>0.05 and x.values[i,4]>0.8 :
        bangers.append([p_id.values[x.values[i,0]-1][1],p_id.values[x.values[i,0]-1][3],v])
print(len(bangers))
print(bangers)
pd.DataFrame(sorted(bangers,key=last,reverse=True)).to_csv("bangers.csv")

23
[['Calbert Cheaney', '1997', 0.0661633983370331], ['Hersey Hawkins', '1997', 0.05669847990867867], ['Juwan Howard', '1997', 0.05310003691683168], ['John Starks', '1997', 0.11839925589745315], ['Shandon Anderson', '1998', 0.05491703250182475], ['Pete Chilcutt', '1998', 0.11336286224545064], ['Travis Knight', '1998', 0.09236418571122228], ['Billy Owens', '1998', 0.07216362998433441], ['Jason Caffey', '2000', 0.06824675339600197], ['Chris Childs', '2000', 0.07457964101805159], ['Vonteego Cummings', '2000', 0.08033721017397256], ['Howard Eisley', '2000', 0.14453196937194057], ['Karl Malone*', '2000', 0.08123874587316167], ['Jalen Rose', '2000', 0.056146174849766584], ['Antonio Daniels', '2001', 0.11765802036676105], ['Brian Grant', '2001', 0.05826710309367519], ['DerMarr Johnson', '2001', 0.06562044093749336], ['Dirk Nowitzki*', '2001', 0.05835240019131743], ['Lamar Odom', '2001', 0.06974980706288518], ['Ruben Patterson', '2001', 0.10535023171150504], ['Darius Miles', '2002', 0.06889583

In [314]:
pd.DataFrame(sorted(bangers,key=last,reverse=True)).to_csv("bangers.csv")

In [315]:
x.cons_mvp=np.zeros_like(x.cons_mvp)

In [327]:
y_pred=lr_model.predict(x_test)
bangers=[]
for i,v in enumerate(y_pred):
    if v>0.1 and x_test.values[i,4]*x_test.values[i,6]>0.4 :
        bangers.append([p_id.values[x_test.values[i,0]-1][1],p_id.values[x_test.values[i,0]-1][3],v])
print(len(bangers))
print(x)
pd.DataFrame(sorted(bangers,key=last,reverse=True)).to_csv("bangers_2014.csv")

8
          Rk       Age      seed  cons_mvp         G        GS        MP  \
0          1  0.346154  0.571429       0.0  0.880952  0.614458  0.649886   
1          2  0.076923  0.928571       0.0  0.940476  0.855422  0.800915   
2          3  0.538462  0.357143       0.0  0.476190  0.036145  0.199085   
3          4  0.192308  0.857143       0.0  0.940476  0.072289  0.416476   
4          5  0.230769  0.642857       0.0  0.892857  0.012048  0.283753   
...      ...       ...       ...       ...       ...       ...       ...   
12841  12842  0.615385  0.571429       0.0  0.630952  0.108434  0.336384   
12842  12843  0.230769  0.500000       0.0  0.857143  0.879518  0.796339   
12843  12844  0.230769  0.428571       0.0  0.095238  0.000000  0.210526   
12844  12845  0.461538  0.428571       0.0  0.166667  0.024096  0.331808   
12845  12846  0.269231  0.285714       0.0  0.892857  0.915663  0.654462   

             FG       FGA    FG%  ...   TOV%      USG%       OWS       DWS  \
0      